# Mandatory 2 - Session 9

Statement:
* Read all pairs of sentences of the trial set within the
evaluation framework of the project.
* Compute the Jaccard similarity of each pair using the
dependency triples from CoreNLPDependencyParser.
* Show the results. Do you think it could be relevant to use
NEs to compute the similarity between two sentences?
Justify the answer

First read the data sets using pandas ...

In [1]:
import pandas as pd
import csv
from nltk.parse.corenlp import CoreNLPDependencyParser
from copy import deepcopy

In [2]:
trial_path    = 'data/trial/STS.input.txt'
trial_gs_path = 'data/trial/STS.gs.txt'
trial_df      = pd.read_csv(trial_path, sep='\t', lineterminator='\n', names=['sentence0','sentence1'], header=None, quoting=csv.QUOTE_NONE)
trial_gs      = pd.read_csv(trial_gs_path, sep='\t', lineterminator='\n', names=['labels'], header=None, quoting=csv.QUOTE_NONE).iloc[::-1]
trial_df

,sentence0,sentence1
id1,The bird is bathing in the sink.,Birdie is washing itself in the water basin.\r
id2,"In May 2010, the troops attempted to invade Ka...",The US army invaded Kabul on May 7th last year...
id3,John said he is considered a witness but not a...,"""He is not a suspect anymore."" John said.\r"
id4,They flew out of the nest in groups.,They flew into the nest together.\r
id5,The woman is playing the violin.,The young lady enjoys listening to the guitar.\r
id6,John went horse back riding at dawn with a who...,Sunrise at dawn is a magnificent view to take ...


Start the CoreNLPDependencyParser

_java -Xmx5g -cp C:\stanford-corenlp-full-2018-10-05\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000_

In [3]:
parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [4]:
def apply_dependency_triplets(sentence):
    print(sentence)
    parse = parser.raw_parse(sentence)
    tree = next(parse)
    for t in tree.triples():
        print(t)

def preprocessing(data):
    ''' Preprocess all sentences to ... '''
    data = data.fillna('')
    for column in data.columns:
        # get triplets
        data[column] = data[column].apply(apply_dependency_triplets)
        #data[column] = data[column].str.lower()
    return data

In [7]:
sentence = "the bird is bathing in the sink"
parse = parser.raw_parse(sentence)
tree = next(parse)
for t in tree.triples():
    print(t)
sentence = "birdie is washing itself in the water basin"
parse = parser.raw_parse(sentence)
tree = next(parse)
for t in tree.triples():
    print(t)

(('bathing', 'NN'), 'nsubj', ('bird', 'NN'))
(('bird', 'NN'), 'det', ('the', 'DT'))
(('bathing', 'NN'), 'cop', ('is', 'VBZ'))
(('bathing', 'NN'), 'nmod', ('sink', 'NN'))
(('sink', 'NN'), 'case', ('in', 'IN'))
(('sink', 'NN'), 'det', ('the', 'DT'))
(('washing', 'VBG'), 'nsubj', ('birdie', 'NN'))
(('washing', 'VBG'), 'aux', ('is', 'VBZ'))
(('washing', 'VBG'), 'dobj', ('itself', 'PRP'))
(('washing', 'VBG'), 'nmod', ('basin', 'NN'))
(('basin', 'NN'), 'case', ('in', 'IN'))
(('basin', 'NN'), 'det', ('the', 'DT'))
(('basin', 'NN'), 'compound', ('water', 'NN'))


In [ ]:
result = preprocessing(deepcopy(trial_df))
result